In [1]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
import time
import sys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import pandas as pd

In [2]:
exeDriver = "chromedriver.exe"
driver = webdriver.Chrome(exeDriver)
print("Iniciando Scraper")

Iniciando Scraper


In [3]:
url = "http://procomun.educalab.es/es"
driver.get(url)

### Busqueda y filtrado por recursos de aprendizaje

In [4]:
content_type = driver.find_element_by_id('edit_type_chosen')
content_type_link = content_type.find_element_by_class_name("chosen-single")
content_type_link.click()

options = content_type.find_elements_by_class_name("active-result")
learning_object = options[1]
learning_object.click()
button = driver.find_element_by_id("new-search")
button.click()
time.sleep(5)
#driver.implicitly_wait(2)

### Obteniendo metadatos articulos

In [5]:
def get_metadata_page():
    URL = driver.current_url
    
    TITULO = driver.find_element_by_xpath('/html/body/section/div/div/div[1]/div/span[5]')
    TITULO = TITULO.text
    
    AUTORES = driver.find_element_by_class_name('author')
    AUTORES = AUTORES.text
    
    FECHA = driver.find_element_by_class_name('detail-ode-publish')
    FECHA = FECHA.text

    PALABRAS_CLAVE = ''
    try:
        general = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[1]/div[1]/div')
        general.click()
        palabras = general.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[1]/div[2]/div/div/div')
        palabras = palabras.text
        palabras_list = palabras.split('\n')
        PALABRAS_CLAVE = palabras_list
    except:
        pass
    
    # Seccion principal
    RESUMEN = ''
    ORIENTACIONES_DIDACTICAS = ''
    LICENCIA = ''
    TIPO_DE_LICENCIA = ''
    USUARIO_FINAL = ''
    CONTEXTO_EDUCATIVO = ''
    seccion_metadatos = driver.find_element_by_id('block-system-main')
    metadatos = seccion_metadatos.find_element_by_xpath('//*[@id="block-system-main"]/div/div/div[2]/div[1]')
    metadatos_txt = metadatos.text
    metadatos_list = metadatos_txt.split('\n')
    for i in range(len(metadatos_list)):
        if (metadatos_list[i] == 'RESUMEN'):
            for j in range(i+1, len(metadatos_list)):
                if ((metadatos_list[j] == 'ORIENTACIONES DIDÁCTICAS') or (metadatos_list[j] == 'LICENCIA')):
                    break
                else:
                    RESUMEN += metadatos_list[j] + ' '
        if (metadatos_list[i] == 'ORIENTACIONES DIDÁCTICAS'):
            for j in range(i+1, len(metadatos_list)):
                if (metadatos_list[j] == 'LICENCIA'):
                    break
                else:
                    ORIENTACIONES_DIDACTICAS += metadatos_list[j] + ' '
        if (metadatos_list[i] == 'LICENCIA'):
            LICENCIA = metadatos_list[i+1]
        if (metadatos_list[i] == 'TIPO DE LICENCIA'):
            TIPO_DE_LICENCIA = metadatos_list[i+1]
        if (metadatos_list[i] == 'USUARIO FINAL'):
            USUARIO_FINAL = metadatos_list[i+1]
        if (metadatos_list[i] == 'CONTEXTO EDUCATIVO'):
            CONTEXTO_EDUCATIVO = metadatos_list[i+1]
        
    # Seccion USO EDUCATIVO 
    USO_EDUCATIVO_EDAD = ''
    USO_EDUCATIVO_IDIOMA_DESTINATARIO = ''
    USO_EDUCATIVO_PROCESO_COGNITIVO = ''
    USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD = ''
    usoEducativo = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[3]/div[1]/div')
    usoEducativo.click()
    elementos = usoEducativo.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[3]')
    elementos = elementos.text
    elementos_list = elementos.split('\n')
    length = len(elementos_list)
    if (length > 1):
        for i in range(1, length):
            if (elementos_list[i] == 'EDAD'):
                USO_EDUCATIVO_EDAD = elementos_list[i+1]
            if (elementos_list[i] == 'IDIOMA DESTINATARIO'):
                USO_EDUCATIVO_IDIOMA_DESTINATARIO = elementos_list[i+1]
            if (elementos_list[i] == 'PROCESO COGNITIVO'):
                USO_EDUCATIVO_PROCESO_COGNITIVO = elementos_list[i+1]
            if (elementos_list[i] == 'TIPO DE INTERACTIVIDAD'):
                USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD = elementos_list[i+1]
                
    # Seccion tecnica
    TECNICA_PLATAFORMA = ''
    try:
        tecnica = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[2]/div[1]/div')
        tecnica.click()
        elementosTecnica = tecnica.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[2]/div[2]/div/div/div')
        TECNICA_PLATAFORMA = elementosTecnica.text
    except:
        pass
    
    # Seccion clasificacion
    CLASIFICACION_AREA_DE_CONOCIMIENTO = ''
    CLASIFICACION_CONTEXTO_EDUCATIVO = ''
    CLASIFICACION_TIPO_DE_RECURSO = ''
    clasificacion = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[4]/div[1]/div')
    clasificacion.click()
    elemClasificacion = clasificacion.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[4]')
    elemClasificacion = elemClasificacion.text
    elemClasificacion_list = elemClasificacion.split('\n')
    length = len(elemClasificacion_list)
    if (length > 1):
        for i in range(1, length):
            if (elemClasificacion_list[i] == 'TIPO DE RECURSO'):
                CLASIFICACION_TIPO_DE_RECURSO = elemClasificacion_list[i+1]
            if (elemClasificacion_list[i] == 'ÁREA DE CONOCIMIENTO'):
                CLASIFICACION_AREA_DE_CONOCIMIENTO = elemClasificacion_list[i+1]
            if (elemClasificacion_list[i] == 'CONTEXTO EDUCATIVO'):
                CLASIFICACION_CONTEXTO_EDUCATIVO = elemClasificacion_list[i+1]

    # Agregando a df
    diccionario = {'TITULO': TITULO, 'PALABRAS_CLAVE': PALABRAS_CLAVE, 'RESUMEN': RESUMEN, 
                   'ORIENTACIONES_DIDACTICAS': ORIENTACIONES_DIDACTICAS, 'LICENCIA': LICENCIA, 
                   'TIPO_DE_LICENCIA': TIPO_DE_LICENCIA, 'AUTORES': AUTORES, 'USUARIO_FINAL': USUARIO_FINAL, 
                   'CONTEXTO_EDUCATIVO': CONTEXTO_EDUCATIVO, 'FECHA': FECHA, 'USO_EDUCATIVO_EDAD': USO_EDUCATIVO_EDAD, 
                   'USO_EDUCATIVO_IDIOMA_DESTINATARIO': USO_EDUCATIVO_IDIOMA_DESTINATARIO, 'USO_EDUCATIVO_PROCESO_COGNITIVO': USO_EDUCATIVO_PROCESO_COGNITIVO,
                   'USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD': USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD, 'URL': URL, 'TECNICA_PLATAFORMA': TECNICA_PLATAFORMA,
                   'CLASIFICACION_AREA_DE_CONOCIMIENTO': CLASIFICACION_AREA_DE_CONOCIMIENTO, 'CLASIFICACION_CONTEXTO_EDUCATIVO': CLASIFICACION_CONTEXTO_EDUCATIVO, 
                   'CLASIFICACION_TIPO_DE_RECURSO': CLASIFICACION_TIPO_DE_RECURSO}
    return diccionario

In [6]:
columns = ['TITULO', 'PALABRAS_CLAVE', 'AUTORES', 'FECHA', 'RESUMEN', 'ORIENTACIONES_DIDACTICAS', 
           'LICENCIA', 'TIPO_DE_LICENCIA', 'USUARIO_FINAL', 'CONTEXTO_EDUCATIVO', 'USO_EDUCATIVO_EDAD',
           'USO_EDUCATIVO_IDIOMA_DESTINATARIO', 'USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD', 
           'USO_EDUCATIVO_PROCESO_COGNITIVO','TECNICA_PLATAFORMA', 'CLASIFICACION_AREA_DE_CONOCIMIENTO', 
           'CLASIFICACION_CONTEXTO_EDUCATIVO', 'CLASIFICACION_TIPO_DE_RECURSO', 'URL']
df = pd.DataFrame(columns=columns)

for i in range(1, 1000): # Itera sobre paginas
    for j in range(1, 11): # Itera sobre articulos
        try:
            articulo1 = driver.find_element_by_xpath('/html/body/section/div/div/div/section/div/section/div[2]/ul/li[' + str(j) + ']/div[1]/h2/a')
            articulo1.click()
            driver.switch_to.window(driver.window_handles[-1])

            diccionario = get_metadata_page()   
            df = df.append(diccionario, ignore_index=True)

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            print('Fallo la pagina ' + str(i) + ' el articulo ' + str(j))
            pass

    try:
        print('Van: ' + str(i) + ' paginas')
        next_page = driver.find_element_by_class_name("pager-next")
        next_page.click()
        time.sleep(5)
        #driver.implicitly_wait(2)
    except:
        print("Fallo al pasar de pagina")
        pass

Van: 1 paginas
Van: 2 paginas
Van: 3 paginas
Van: 4 paginas
Fallo la pagina 5 el articulo 1
Fallo la pagina 5 el articulo 2
Fallo la pagina 5 el articulo 3
Fallo la pagina 5 el articulo 4
Fallo la pagina 5 el articulo 5
Fallo la pagina 5 el articulo 6
Fallo la pagina 5 el articulo 7
Fallo la pagina 5 el articulo 8
Fallo la pagina 5 el articulo 9
Fallo la pagina 5 el articulo 10
Van: 5 paginas
Fallo al pasar de pagina
Fallo la pagina 6 el articulo 1
Fallo la pagina 6 el articulo 2
Fallo la pagina 6 el articulo 3
Fallo la pagina 6 el articulo 4
Fallo la pagina 6 el articulo 5
Fallo la pagina 6 el articulo 6
Fallo la pagina 6 el articulo 7
Fallo la pagina 6 el articulo 8
Van: 6 paginas
Van: 7 paginas
Van: 8 paginas
Van: 9 paginas
Van: 10 paginas
Van: 11 paginas
Van: 12 paginas
Van: 13 paginas
Van: 14 paginas
Van: 15 paginas
Van: 16 paginas
Van: 17 paginas
Van: 18 paginas
Van: 19 paginas
Van: 20 paginas
Van: 21 paginas
Van: 22 paginas
Van: 23 paginas
Van: 24 paginas
Van: 25 paginas
Van: 26

Van: 353 paginas
Van: 354 paginas
Van: 355 paginas
Van: 356 paginas
Van: 357 paginas
Van: 358 paginas
Van: 359 paginas
Van: 360 paginas
Van: 361 paginas
Van: 362 paginas
Van: 363 paginas
Van: 364 paginas
Van: 365 paginas
Van: 366 paginas
Van: 367 paginas
Van: 368 paginas
Van: 369 paginas
Van: 370 paginas
Van: 371 paginas
Van: 372 paginas
Van: 373 paginas
Van: 374 paginas
Van: 375 paginas
Van: 376 paginas
Van: 377 paginas
Van: 378 paginas
Van: 379 paginas
Van: 380 paginas
Van: 381 paginas
Van: 382 paginas
Van: 383 paginas
Van: 384 paginas
Van: 385 paginas
Van: 386 paginas
Van: 387 paginas
Van: 388 paginas
Van: 389 paginas
Van: 390 paginas
Van: 391 paginas
Van: 392 paginas
Van: 393 paginas
Van: 394 paginas
Van: 395 paginas
Van: 396 paginas
Van: 397 paginas
Van: 398 paginas
Van: 399 paginas
Van: 400 paginas
Van: 401 paginas
Van: 402 paginas
Van: 403 paginas
Van: 404 paginas
Van: 405 paginas
Van: 406 paginas
Van: 407 paginas
Van: 408 paginas
Van: 409 paginas
Van: 410 paginas
Van: 411 pagin

Fallo al pasar de pagina
Fallo la pagina 492 el articulo 1
Fallo la pagina 492 el articulo 2
Fallo la pagina 492 el articulo 3
Fallo la pagina 492 el articulo 4
Fallo la pagina 492 el articulo 5
Fallo la pagina 492 el articulo 6
Fallo la pagina 492 el articulo 7
Fallo la pagina 492 el articulo 8
Fallo la pagina 492 el articulo 9
Fallo la pagina 492 el articulo 10
Van: 492 paginas
Fallo al pasar de pagina
Fallo la pagina 493 el articulo 1
Fallo la pagina 493 el articulo 2
Fallo la pagina 493 el articulo 3
Fallo la pagina 493 el articulo 4
Fallo la pagina 493 el articulo 5
Fallo la pagina 493 el articulo 6
Fallo la pagina 493 el articulo 7
Fallo la pagina 493 el articulo 8
Fallo la pagina 493 el articulo 9
Fallo la pagina 493 el articulo 10
Van: 493 paginas
Fallo al pasar de pagina
Fallo la pagina 494 el articulo 1
Fallo la pagina 494 el articulo 2
Fallo la pagina 494 el articulo 3
Fallo la pagina 494 el articulo 4
Fallo la pagina 494 el articulo 5
Fallo la pagina 494 el articulo 6
Fallo

Fallo la pagina 514 el articulo 3
Fallo la pagina 514 el articulo 4
Fallo la pagina 514 el articulo 5
Fallo la pagina 514 el articulo 6
Fallo la pagina 514 el articulo 7
Fallo la pagina 514 el articulo 8
Fallo la pagina 514 el articulo 9
Fallo la pagina 514 el articulo 10
Van: 514 paginas
Fallo al pasar de pagina
Fallo la pagina 515 el articulo 1
Fallo la pagina 515 el articulo 2
Fallo la pagina 515 el articulo 3
Fallo la pagina 515 el articulo 4
Fallo la pagina 515 el articulo 5
Fallo la pagina 515 el articulo 6
Fallo la pagina 515 el articulo 7
Fallo la pagina 515 el articulo 8
Fallo la pagina 515 el articulo 9
Fallo la pagina 515 el articulo 10
Van: 515 paginas
Fallo al pasar de pagina
Fallo la pagina 516 el articulo 1
Fallo la pagina 516 el articulo 2
Fallo la pagina 516 el articulo 3
Fallo la pagina 516 el articulo 4
Fallo la pagina 516 el articulo 5
Fallo la pagina 516 el articulo 6
Fallo la pagina 516 el articulo 7
Fallo la pagina 516 el articulo 8
Fallo la pagina 516 el articul

Fallo la pagina 537 el articulo 7
Fallo la pagina 537 el articulo 8
Fallo la pagina 537 el articulo 9
Fallo la pagina 537 el articulo 10
Van: 537 paginas
Fallo al pasar de pagina
Fallo la pagina 538 el articulo 1
Fallo la pagina 538 el articulo 2
Fallo la pagina 538 el articulo 3
Fallo la pagina 538 el articulo 4
Fallo la pagina 538 el articulo 5
Fallo la pagina 538 el articulo 6
Fallo la pagina 538 el articulo 7
Fallo la pagina 538 el articulo 8
Fallo la pagina 538 el articulo 9
Fallo la pagina 538 el articulo 10
Van: 538 paginas
Fallo al pasar de pagina
Fallo la pagina 539 el articulo 1
Fallo la pagina 539 el articulo 2
Fallo la pagina 539 el articulo 3
Fallo la pagina 539 el articulo 4
Fallo la pagina 539 el articulo 5
Fallo la pagina 539 el articulo 6
Fallo la pagina 539 el articulo 7
Fallo la pagina 539 el articulo 8
Fallo la pagina 539 el articulo 9
Fallo la pagina 539 el articulo 10
Van: 539 paginas
Fallo al pasar de pagina
Fallo la pagina 540 el articulo 1
Fallo la pagina 540 e

...

Fallo la pagina 892 el articulo 7
Fallo la pagina 892 el articulo 8
Fallo la pagina 892 el articulo 9
Fallo la pagina 892 el articulo 10
Van: 892 paginas
Fallo al pasar de pagina
Fallo la pagina 893 el articulo 1
Fallo la pagina 893 el articulo 2
Fallo la pagina 893 el articulo 3
Fallo la pagina 893 el articulo 4
Fallo la pagina 893 el articulo 5
Fallo la pagina 893 el articulo 6
Fallo la pagina 893 el articulo 7
Fallo la pagina 893 el articulo 8
Fallo la pagina 893 el articulo 9
Fallo la pagina 893 el articulo 10
Van: 893 paginas
Fallo al pasar de pagina
Fallo la pagina 894 el articulo 1
Fallo la pagina 894 el articulo 2
Fallo la pagina 894 el articulo 3
Fallo la pagina 894 el articulo 4
Fallo la pagina 894 el articulo 5
Fallo la pagina 894 el articulo 6
Fallo la pagina 894 el articulo 7
Fallo la pagina 894 el articulo 8
Fallo la pagina 894 el articulo 9
Fallo la pagina 894 el articulo 10
Van: 894 paginas
Fallo al pasar de pagina
Fallo la pagina 895 el articulo 1
Fallo la pagina 895 e

Fallo la pagina 914 el articulo 4
Fallo la pagina 914 el articulo 5
Fallo la pagina 914 el articulo 6
Fallo la pagina 914 el articulo 7
Fallo la pagina 914 el articulo 8
Fallo la pagina 914 el articulo 9
Fallo la pagina 914 el articulo 10
Van: 914 paginas
Fallo al pasar de pagina
Fallo la pagina 915 el articulo 1
Fallo la pagina 915 el articulo 2
Fallo la pagina 915 el articulo 3
Fallo la pagina 915 el articulo 4
Fallo la pagina 915 el articulo 5
Fallo la pagina 915 el articulo 6
Fallo la pagina 915 el articulo 7
Fallo la pagina 915 el articulo 8
Fallo la pagina 915 el articulo 9
Fallo la pagina 915 el articulo 10
Van: 915 paginas
Fallo al pasar de pagina
Fallo la pagina 916 el articulo 1
Fallo la pagina 916 el articulo 2
Fallo la pagina 916 el articulo 3
Fallo la pagina 916 el articulo 4
Fallo la pagina 916 el articulo 5
Fallo la pagina 916 el articulo 6
Fallo la pagina 916 el articulo 7
Fallo la pagina 916 el articulo 8
Fallo la pagina 916 el articulo 9
Fallo la pagina 916 el articul

Fallo al pasar de pagina
Fallo la pagina 937 el articulo 1
Fallo la pagina 937 el articulo 2
Fallo la pagina 937 el articulo 3
Fallo la pagina 937 el articulo 4
Fallo la pagina 937 el articulo 5
Fallo la pagina 937 el articulo 6
Fallo la pagina 937 el articulo 7
Fallo la pagina 937 el articulo 8
Fallo la pagina 937 el articulo 9
Fallo la pagina 937 el articulo 10
Van: 937 paginas
Fallo al pasar de pagina
Fallo la pagina 938 el articulo 1
Fallo la pagina 938 el articulo 2
Fallo la pagina 938 el articulo 3
Fallo la pagina 938 el articulo 4
Fallo la pagina 938 el articulo 5
Fallo la pagina 938 el articulo 6
Fallo la pagina 938 el articulo 7
Fallo la pagina 938 el articulo 8
Fallo la pagina 938 el articulo 9
Fallo la pagina 938 el articulo 10
Van: 938 paginas
Fallo al pasar de pagina
Fallo la pagina 939 el articulo 1
Fallo la pagina 939 el articulo 2
Fallo la pagina 939 el articulo 3
Fallo la pagina 939 el articulo 4
Fallo la pagina 939 el articulo 5
Fallo la pagina 939 el articulo 6
Fallo

Fallo la pagina 958 el articulo 7
Fallo la pagina 958 el articulo 8
Fallo la pagina 958 el articulo 9
Fallo la pagina 958 el articulo 10
Van: 958 paginas
Fallo al pasar de pagina
Fallo la pagina 959 el articulo 1
Fallo la pagina 959 el articulo 2
Fallo la pagina 959 el articulo 3
Fallo la pagina 959 el articulo 4
Fallo la pagina 959 el articulo 5
Fallo la pagina 959 el articulo 6
Fallo la pagina 959 el articulo 7
Fallo la pagina 959 el articulo 8
Fallo la pagina 959 el articulo 9
Fallo la pagina 959 el articulo 10
Van: 959 paginas
Fallo al pasar de pagina
Fallo la pagina 960 el articulo 1
Fallo la pagina 960 el articulo 2
Fallo la pagina 960 el articulo 3
Fallo la pagina 960 el articulo 4
Fallo la pagina 960 el articulo 5
Fallo la pagina 960 el articulo 6
Fallo la pagina 960 el articulo 7
Fallo la pagina 960 el articulo 8
Fallo la pagina 960 el articulo 9
Fallo la pagina 960 el articulo 10
Van: 960 paginas
Fallo al pasar de pagina
Fallo la pagina 961 el articulo 1
Fallo la pagina 961 e

Fallo al pasar de pagina
Fallo la pagina 981 el articulo 1
Fallo la pagina 981 el articulo 2
Fallo la pagina 981 el articulo 3
Fallo la pagina 981 el articulo 4
Fallo la pagina 981 el articulo 5
Fallo la pagina 981 el articulo 6
Fallo la pagina 981 el articulo 7
Fallo la pagina 981 el articulo 8
Fallo la pagina 981 el articulo 9
Fallo la pagina 981 el articulo 10
Van: 981 paginas
Fallo al pasar de pagina
Fallo la pagina 982 el articulo 1
Fallo la pagina 982 el articulo 2
Fallo la pagina 982 el articulo 3
Fallo la pagina 982 el articulo 4
Fallo la pagina 982 el articulo 5
Fallo la pagina 982 el articulo 6
Fallo la pagina 982 el articulo 7
Fallo la pagina 982 el articulo 8
Fallo la pagina 982 el articulo 9
Fallo la pagina 982 el articulo 10
Van: 982 paginas
Fallo al pasar de pagina
Fallo la pagina 983 el articulo 1
Fallo la pagina 983 el articulo 2
Fallo la pagina 983 el articulo 3
Fallo la pagina 983 el articulo 4
Fallo la pagina 983 el articulo 5
Fallo la pagina 983 el articulo 6
Fallo

In [7]:
df

,TITULO,PALABRAS_CLAVE,AUTORES,FECHA,RESUMEN,ORIENTACIONES_DIDACTICAS,LICENCIA,TIPO_DE_LICENCIA,USUARIO_FINAL,CONTEXTO_EDUCATIVO,USO_EDUCATIVO_EDAD,USO_EDUCATIVO_IDIOMA_DESTINATARIO,USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD,USO_EDUCATIVO_PROCESO_COGNITIVO,TECNICA_PLATAFORMA,CLASIFICACION_AREA_DE_CONOCIMIENTO,CLASIFICACION_CONTEXTO_EDUCATIVO,CLASIFICACION_TIPO_DE_RECURSO,URL
0,Habilidades Sociales. Manual de Formación y Cu...,"[FP, educación infantil, habilidades sociales]",JOSE ANGEL DEL POZO FLOREZ,07/10/2019,"Este manual desarrolla el modulo profesional ""...",1.- Implementa de estrategias y técnicas que f...,Creative Commons: Reconocimiento - Compartir i...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,,español (o castellano),,,,"Ciencias de la Educación, Orientación Académic...",Ciclo formativo grado superior (más de 18 años),Basado en proyectos,http://procomun.educalab.es/es/ode/view/155912...
1,Implementación de algoritmos,,LAURA MAYA,07/10/2019,"Identificar el concepto, características y tip...",CONOCIMIENTO PREVIO: Software y hardware OBJET...,Creative Commons: Reconocimiento - Compartir i...,Universal,alumno,Presencial Aula,,español (o castellano),,,editor: eXe Learning,,,Basado en proyectos,http://procomun.educalab.es/es/ode/view/155912...
2,epidemiologia_en_salud_oral_,"[Epidemiología, salud publica]",Juan Formigós Bolea,06/10/2019,Apuntes en formato libro pdf del módulo (=asig...,NO se requieren conocimientos previos. El módu...,Creative Commons: Reconocimiento - Compartir i...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,,español (o castellano),,,,Educación Sexual y para la Salud,Ciclo formativo grado superior (más de 18 años),"Texto narrativo, Lección magistral",http://procomun.educalab.es/es/ode/view/155912...
3,Extraescolares Clarín con pictos,"[educación, diversidad, inclusión, accesibilid...",Violeta Iglesias Prieto,05/10/2019,Proyecto de señalización para el desarrollo de...,,Creative Commons: Reconocimiento - Compartir i...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,0-6 6-12,español (o castellano),,,,"Ciencias de la Educación, Necesidades educativ...","Educación Especial, Educación Infantil, Educac...",Basado en proyectos,http://procomun.educalab.es/es/ode/view/155912...
4,Italiano multimedia para españoles,,CAROLINA BARDELLI,04/10/2019,Curso básico orientado a todas aquellas person...,,Creative Commons: Reconocimiento - Compartir i...,Universal,,,,español (o castellano),,,editor: eXe Learning,,,Simulación,http://procomun.educalab.es/es/ode/view/155912...
5,"""APPrendizaje personalizado con TIC"". Paisajes...","[paisajes de aprendizaje, personalización, TIC...",Irene Arrimadas Gómez,27/09/2019,"Resumen del proyecto ""APPrendizaje personaliza...",,Creative Commons: Reconocimiento - No comercia...,Universal,alumno con necesidades educativas especiales g...,Aula Laboratorio Mixta Presencial,,español (o castellano),,,,Necesidades educativas especiales,Educación Especial,"Texto narrativo, Basado en proyectos",http://procomun.educalab.es/es/ode/view/155912...
6,les_veus_dahir_i_davui,"[democracia, mujer, feminismo, movimientos soc...",FELIPE PÉREZ FERRER,26/09/2019,Breve proyecto que trabaja la memoria democrát...,,Creative Commons: Reconocimiento - No comercia...,Universal,alumno con altas capacidades grupal,Aula Laboratorio Mixta Semipresencial,15-16,valenciano,,,,Historia,15 - 16 / Cuarto curso,"Texto narrativo, Basado en proyectos",http://procomun.educalab.es/es/ode/view/155912...
7,actividades grupos interactivos,"[Favorecer la interacción social, al oralidad,...",irene gonzalez collado,23/09/2019,Batería de actividades para desarrollar en los...,,Creative Commons: Reconocimiento - Compartir i...,Universal,tutor grupal,Aula Laboratorio Mixta Presencial,0-6 6-12,español (o castellano),,,,"Educación Ambiental, Lengua, Lenguas Extranjer...","Educación Infantil, Educación Primaria",Juego didáctico,http://procomun.educalab.es/es/ode/view/155912...
8,¿Truco o trato? ¿

In [9]:
df.to_excel("dataEXCEL.xlsx", "dataEXCEL")
try:
    df.to_csv("data.csv", sep = ';', encoding = 'ISO-8859-1')
except:
    df.to_csv("dataNoEncode.csv", sep = ';')